In [1]:
import os
import pandas as pd

In [2]:
# Set the folder where all monthly CPI files are stored
folder_path = "path"  # update this path to your folder

In [3]:
# Container for cleaned data
all_months = []

In [4]:
# Loop through each Excel file in the folder
for file in sorted(os.listdir(folder_path)):
    if file.lower().endswith((".xls", ".xlsx")):
        print(f"Processing file: {file}")
        try:
            # Extract month and year
            base_name = file.replace(".xlsx", "")
            month_str, year_str = base_name.split("_")
            file_date = pd.to_datetime(f"{year_str}-{month_str}")

            file_path = os.path.join(folder_path, file)
            df = pd.read_excel(file_path, skiprows=6)

            df = df.rename(columns={
                df.columns[1]: "SubGroup_Code",
                df.columns[2]: "Description",
                df.columns[9]: "Weight", 
                df.columns[11]: "Provisional_CPI"
            })

            # Drop non-subgroup rows
            df = df[~df["SubGroup_Code"].isna() & ~df["Description"].isna()]
            df = df[pd.to_numeric(df["Provisional_CPI"], errors="coerce").notna()]

            df_clean = df[["SubGroup_Code", "Description", "Weight", "Provisional_CPI"]].copy()
            df_clean["Date"] = pd.to_datetime(file_date.strftime('%Y-%m-%d'), format='%Y-%m-%d')
            df_clean = df_clean.rename(columns={"Provisional_CPI": "CPI"})

            all_months.append(df_clean)
            print(f"Successfully processed {file}")

        except Exception as e:
            print(f"Error processing {file}: {e}")


Processing file: 01_2017.xlsx
Successfully processed 01_2017.xlsx
Processing file: 01_2018.xlsx
Successfully processed 01_2018.xlsx
Processing file: 01_2019.xlsx
Successfully processed 01_2019.xlsx
Processing file: 01_2020.xlsx
Successfully processed 01_2020.xlsx
Processing file: 01_2021.xlsx
Successfully processed 01_2021.xlsx
Processing file: 01_2022.xlsx
Successfully processed 01_2022.xlsx
Processing file: 01_2023.xlsx
Successfully processed 01_2023.xlsx
Processing file: 01_2024.xlsx
Successfully processed 01_2024.xlsx
Processing file: 01_2025.xlsx
Successfully processed 01_2025.xlsx
Processing file: 02_2017.xlsx
Successfully processed 02_2017.xlsx
Processing file: 02_2018.xlsx
Successfully processed 02_2018.xlsx
Processing file: 02_2019.xlsx
Successfully processed 02_2019.xlsx
Processing file: 02_2020.xlsx
Successfully processed 02_2020.xlsx
Processing file: 02_2021.xlsx
Successfully processed 02_2021.xlsx
Processing file: 02_2022.xlsx
Successfully processed 02_2022.xlsx
Processing

In [6]:
# Concatenate all monthly data
final_df = pd.concat(all_months, ignore_index=True)

# Sort by date and subgroup
final_df = final_df.sort_values(by=["SubGroup_Code", "Date"])

In [9]:
# Output to CSV
final_df.to_csv("F:/ML/CPI/Weighted_CPI_Forecasting/combined_cpi_data.csv", index=False)